In [9]:
import pandas as pd

df = pd.read_csv('real_news.csv', encoding='ISO-8859-1')
df


,id,category,topic,source,headline,text,link
0,4,True,Education,EL UNIVERSAL,UNAM capacitarï¿½ a maestros para aprobar prue...,UNAM capacitarï¿½ a maestros para aprobar prue...,http://www.eluniversal.com.mx/articulo/nacion/...
1,6,True,Education,Heraldo,Un paso mï¿½s cerca de hacer los exï¿½menes 'o...,Un paso mï¿½s cerca de hacer los exï¿½menes 'o...,https://www.heraldo.es/noticias/suplementos/he...
2,8,True,Education,Excelsior,Niï¿½o de *NUMBER* aï¿½os se prepara para entr...,Niï¿½o de *NUMBER* aï¿½os se prepara para entr...,https://www.excelsior.com.mx/trending/nino-de-...
3,9,True,Education,El paï¿½s,*NUMBER* palabras que creï¿½amos inaceptables ...,*NUMBER* palabras que creï¿½amos inaceptables ...,https://elpais.com/elpais/2018/03/28/icon/1522...
4,11,True,Education,Excelsior,La UNAM limpia aulas de alumnos rezagados,La UNAM limpia aulas de alumnos rezagados\nA p...,https://www.excelsior.com.mx/nacional/2015/09/...
...,...,...,...,...,...,...,...
619,562,True,Sociedad,El Paï¿½s,Detenidos seis menores por supuestos abusos a ...,Seis menores han sido detenidos en Cï¿½diz acu...,https://elpais.com/politica/2017/09/12/actuali...
620,563,True,Sociedad,El Comercio,La falsa noticia de la empleada que defeca en ...,Muchos quisieron creer que fuera verdad al con...,https://www.elcomercio.es/sociedad/201604/30/f...
621,567,True,Sociedad,El Tiempo,Las razones detrï¿½s del aumento de suicidios ...,"Durante los ï¿½ltimos cuatro aï¿½os, la ciudad...",https://www.eltiempo.com/colombia/medellin/raz...
622,568,True,Covid-19,El Financiero,Encuentran nueva variante de COVID en Mï¿½xico...,El Instituto de Diagnï¿½stico y Referencia Epi...,https://www.elfinanciero.com.mx/salud/encuentr...


In [10]:
import csv
import openai

openai.api_key = "key"

def generate_summary_csv(input_file, output_file, max_tokens=60, window_size=100, overlap=20):
    with open(input_file, 'r', encoding='utf-8') as f_in, open(output_file, 'w', encoding='utf-8', newline='') as f_out:
        reader = csv.reader(f_in)
        writer = csv.writer(f_out)

        # iterate over rows in input file
        for i, row in enumerate(reader):
            if i == 0:
                # header row, write header to output file
                writer.writerow(row + ["summary"])
            else:
                # extract text from appropriate column in row
                text = row[5]  # replace\ with the index of the column containing the text

                # split text into overlapping windows of size window_size
                chunks = [text[i:i+window_size] for i in range(0, len(text), window_size-overlap)]

                # generate summaries for each chunk
                summaries = []
                for chunk in chunks:
                    response = openai.Completion.create(
                        engine="davinci",
                        prompt=f"Summarize the following text into Spanish into {max_tokens} words or less:\n{chunk}\n",
                        max_tokens=max_tokens,
                        n=1,
                        stop=None,
                        temperature=0.5,
                    )
                    summary = response.choices[0].text.strip()
                    summaries.append(summary)

                # concatenate the summaries and write to output file
                writer.writerow(row + [" ".join(summaries)])

# example usage
input_file = "real_news.csv"
output_file = "real_news_output.csv"

generate_summary_csv(input_file, output_file, max_tokens=60, window_size=100, overlap=20)


In [11]:
summaries = pd.read_csv('real_news_output.csv', encoding='utf-8')
summaries

,id,category,topic,source,headline,text,link,summary
0,4,True,Education,EL UNIVERSAL,UNAM capacitar� a maestros para aprobar prueba...,UNAM capacitar� a maestros para aprobar prueba...,http://www.eluniversal.com.mx/articulo/nacion/...,UNAM capacitar� a maestros para aprobar prueba...
1,6,True,Education,Heraldo,Un paso m�s cerca de hacer los ex�menes 'online',Un paso m�s cerca de hacer los ex�menes 'onlin...,https://www.heraldo.es/noticias/suplementos/he...,Vocabulary:\nUniversidades - Universities\nEx�...
2,8,True,Education,Excelsior,Ni�o de *NUMBER* a�os se prepara para entrar a...,Ni�o de *NUMBER* a�os se prepara para entrar a...,https://www.excelsior.com.mx/trending/nino-de-...,"Include a title, a summary, and a brief descri..."
3,9,True,Education,El pa�s,*NUMBER* palabras que cre�amos inaceptables y ...,*NUMBER* palabras que cre�amos inaceptables y ...,https://elpais.com/elpais/2018/03/28/icon/1522...,*NUMBER* palabras que cre�amos inaceptables y ...
4,11,True,Education,Excelsior,La UNAM limpia aulas de alumnos rezagados,La UNAM limpia aulas de alumnos rezagados\nA p...,https://www.excelsior.com.mx/nacional/2015/09/...,nencia en la UNAM establecen que los alumnos q...
...,...,...,...,...,...,...,...,...
619,562,True,Sociedad,El Pa�s,Detenidos seis menores por supuestos abusos a ...,Seis menores han sido detenidos en C�diz acusa...,https://elpais.com/politica/2017/09/12/actuali...,un menor de nueve a�os en una piscina municipa...
620,563,True,Sociedad,El Comercio,La falsa noticia de la empleada que defeca en ...,Muchos quisieron creer que fuera verdad al con...,https://www.elcomercio.es/sociedad/201604/30/f...,fue as�. El hombre que los llev� a la cima del...
621,567,True,Sociedad,El Tiempo,Las razones detr�s del aumento de suicidios en...,"Durante los �ltimos cuatro a�os, la ciudad no ...",https://www.eltiempo.com/colombia/medellin/raz...,a tasa de suicidios en la ciudad aument� en un...
622,568,True,Covid-19,El Financiero,Encuentran nueva variante de COVID en M�xico: ...,El Instituto de Diagn�stico y Referencia Epide...,https://www.elfinanciero.com.mx/salud/encuentr...,aumento de casos de c�ncer en la poblaci�n de ...
